In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from sentence_transformers import SentenceTransformer, util
import torch
from dotenv import load_dotenv
from mongo_wrapper.mongo_wrapper import MongoWrapper

# states to categorize
STATES = [
    "Guanajuato",
    "Michoacán",
    "Sinaloa",
    "Chihuahua",
    "Guerrero",
    "Tamaulipas",
    "Baja California",
    "Zacatecas",
    "Colima",
    "Jalisco",
    "Aguascalientes",
    "Baja California Sur",
    "Campeche",
    "Coahuila",
    "Durango",
    "Hidalgo",
    "Estado de México",
    "Ciudad de México",
    "Morelos",
    "Nayarit",
    "Nuevo León",
    "Oaxaca",
    "Puebla",
    "Querétaro",
    "Quintana Roo",
    "San Luis Potosí",
    "Sonora",
    "Tabasco",
    "Tlaxcala",
    "Veracruz",
    "Yucatán",
    "Chiapas"]


# channels to analyze 
TARGET_CHANNELS = [
    "elpaismexico",
    "ElUniversalOnline",
    "proceso_unofficial",
    "politicomx",
    "lajornada_unofficial",
    "larazondemexico",
    "sinembargomx",
    "elpaisamerica"]

# define dimensions of poverty 
POVERTY_DIMENSIONS = {
    "INCOME": """
    desempleo, salario mínimo, bajos ingresos, deudas familiares, pobreza laboral, 
    pérdida de empleo, ingreso insuficiente, precariedad laboral, empleo informal, 
    falta de oportunidades laborales, reducción de salario, inestabilidad económica, 
    recesión, subempleo, despidos masivos, contratos temporales, informalidad, 
    costos de vida elevados, falta de empleo formal, insuficiencia salarial
    """,

    "ACCESS TO HEALTH SERVICES": """
    falta de acceso a servicios de salud, hospitales saturados, escasez de medicamentos, 
    deficiencias en la atención médica, carencia de personal médico, emergencia sanitaria, 
    costos elevados de tratamientos, cierre de centros de salud, lista de espera prolongada, 
    equipos médicos inoperantes, desabasto de vacunas, falta de atención especializada, 
    dificultades para obtener citas médicas, crisis hospitalaria, cobertura médica limitada
    """,

    "EDUCATIONAL_LAG": """
    deserción escolar, suspensión de clases, carencia de docentes, infraestructura educativa deficiente, 
    dificultades de acceso a la educación, brecha digital, educación interrumpida, rezago académico, 
    falta de recursos escolares, inasistencia por motivos económicos, limitaciones en conectividad, 
    acceso desigual a la educación, deficiencias en formación básica, carencia de materiales educativos
    """,

    "ACCESS TO SOCIAL SECURITY": """
    empleo informal, ausencia de prestaciones sociales, falta de contrato laboral, 
    exclusión del sistema de pensiones, carencia de protección social, trabajo precario, 
    derechos laborales no garantizados, falta de cotización al sistema, desprotección estructural, 
    dificultades para acceder al seguro social, informalidad laboral, empleo sin afiliación
    """,

    "HOUSING": """
    vivienda precaria, hacinamiento, falta de servicios básicos, 
    infraestructura deteriorada, zonas marginadas, viviendas inseguras
    """,
    
    "ACCESS TO FOOD": """
    inseguridad alimentaria, acceso limitado a alimentos, inflación, precios, 
    raciones insuficientes, pobreza alimentaria, aumento de precios, comeder comunitario, 
    canasta básica, crisis alimentaria, alza de precios, ayuda alimentaria, 
    insuficiencia nutricional, alimentación deficiente, encarecimiento de alimentos,
    inflación en alimentos, carencia alimentaria, productos básicos, banco de alimentos, 
    alimentos inaccesibles, gasto alimentario elevado, programas alimentarios,
    """,

    "SOCIAL_COHESION": """
    discriminación étnica, marginación social, exclusión de comunidades vulnerables, 
    conflictos intercomunitarios, tensiones sociales, barreras sociales, 
    desigualdad estructural, aislamiento social
    """}


class PovertyDimensionClassifier:
    def __init__(self):
        self.model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')
        self.dimension_embeddings = {}
        
        for dim_name, dim_terms in POVERTY_DIMENSIONS.items():
            terms = [t.strip() for t in dim_terms.split(',') if t.strip()]
            term_embeddings = self.model.encode(terms, convert_to_tensor=True)
            avg_embedding = torch.mean(term_embeddings, dim=0)  
            self.dimension_embeddings[dim_name] = avg_embedding
    
    def clean_text(self, text):
        if not isinstance(text, str): 
            return ""
        text = re.sub(r'<.*?>', ' ', text)
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'[^\w\sáéíóúüñÁÉÍÓÚÜÑ]', ' ', text)
        return re.sub(r'\s+', ' ', text).strip().lower()
    
    def classify_text(self, text, threshold=0.10):
        if not text:
            return None, 0.0
        
        cleaned_text = self.clean_text(text)
        if len(cleaned_text) < 10:
            return None, 0.0
        
        text_embedding = self.model.encode(cleaned_text, convert_to_tensor=True)
        
        max_score = -1
        best_dim = None
        
        for dim_name, dim_embedding in self.dimension_embeddings.items():
            text_embed_2d = text_embedding.unsqueeze(0)
            dim_embed_2d = dim_embedding.unsqueeze(0)
            
            cos_sim = util.cos_sim(text_embed_2d, dim_embed_2d).item()
            
            if cos_sim > max_score and cos_sim >= threshold:
                max_score = cos_sim
                best_dim = dim_name
        
        return (best_dim, max_score) if best_dim else (None, max_score)

# load only data of interest from MongoDB
def load_state_posts():
    
    MONGO_IP = os.getenv("MONGO_IP")
    MONGO_PORT = os.getenv("MONGO_PORT")
    MONGO_DB = os.getenv("MONGO_DB")
    MONGO_USERNAME = os.getenv("MONGO_USERNAME")
    MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
    
    mongo_client = MongoWrapper(
        db=MONGO_DB,
        user=MONGO_USERNAME,
        password=MONGO_PASSWORD,
        ip=MONGO_IP,
        port=MONGO_PORT)
    
    all_channels = mongo_client.get_all_collections()
    available_target_channels = [channel for channel in TARGET_CHANNELS if channel in all_channels]
    
    # initialize a dictionary to store posts for each state
    state_posts = {state: [] for state in STATES}
    
    # classify posts by using regex patterns - if a post contains a state name, it will be classified as that state
    state_patterns = {state: re.compile(r'\b' + re.escape(state) + r'\b', re.IGNORECASE) for state in STATES}
    
    for channel in tqdm(available_target_channels, desc="loading channels"):
        posts = mongo_client.get_collection_entries(collection=channel)
        
        print(f"channel: {channel} - {len(posts)} post found")
        
        for post in tqdm(posts, desc=f"analysis {channel}", leave=False):
            post_text = post.get('text', '')
                
            for state, pattern in state_patterns.items():
                if pattern.search(post_text):
                    state_posts[state].append({
                        'text': post_text,
                        'author': post.get('author', ''),
                        'posting_ts': post.get('posting_ts', ''),
                        'channel': channel})
    
    for state in STATES:
        state_posts[state] = pd.DataFrame(state_posts[state])
    
    return state_posts

# classify posts by dimensions of poverty 
def analyze_poverty_dimensions(state_posts):
    classifier = PovertyDimensionClassifier()
    
    results = []

    for state, df in state_posts.items():
        print(f"\nanalyzing {state} ({len(df)} posts)...")
    
        dimension_counts = {dim: 0 for dim in POVERTY_DIMENSIONS.keys()}
        dimension_counts["OTHER"] = 0  # other types of posts 
    
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Classifying {state}"):
            text = row['text']
        
            dimension, score = classifier.classify_text(text)
        
            if dimension:
                dimension_counts[dimension] += 1
            else:
                dimension_counts["OTHER"] += 1
        
        total_posts = len(df)
        dimension_percentages = {dim: (count / total_posts) * 100 for dim, count in dimension_counts.items()}
        
        print(f"\nresults for {state}:")
        print(f"total posts: {total_posts}")
        print("\ndistribution of posts across dimensions of poverty:")
        
        for dim, count in dimension_counts.items():
            dim_name = dim if dim != "OTHER" else "non-poverty posts"
            pct = dimension_percentages[dim]
            print(f"- {dim_name}: {count} post ({pct:.1f}%)")
        
        for dim in list(POVERTY_DIMENSIONS.keys()) + ["OTHER"]:
            results.append({
                'state': state,
                'dimension': dim,
                'count': dimension_counts[dim],
                'percentage': dimension_percentages[dim],
                'total_posts': total_posts})
    
    results_df = pd.DataFrame(results)
    return results_df

def main():
    state_posts = load_state_posts()
    
    results = analyze_poverty_dimensions(state_posts)
    
    results.to_csv("tg_results_improved.csv", index=False)
    
    pivot_counts = results.pivot(index='state', columns='dimension', values='count')
    print("\ncount of posts per dimension:")
    print(pivot_counts)
    
    pivot_percentages = results.pivot(index='state', columns='dimension', values='percentage')
    print("\npercentage of posts per dimension:")
    print(pivot_percentages.round(1))

if __name__ == "__main__":
    main()

2025-05-12 11:14:10,564 WARNING Logger Mongo was configured with True console stream
2025-05-12 11:14:10,968 INFO Connected to thesis database on 206.81.16.39
loading channels:   0%|          | 0/8 [00:00<?, ?it/s]

channel: elpaismexico - 1750 post found


loading channels:  12%|█▎        | 1/8 [00:00<00:04,  1.63it/s]

channel: ElUniversalOnline - 2435 post found


loading channels:  25%|██▌       | 2/8 [00:02<00:06,  1.13s/it]

...


results for Veracruz:
total posts: 264

distribution of posts across dimensions of poverty:
- INCOME: 72 post (27.3%)
- ACCESS TO HEALTH SERVICES: 52 post (19.7%)
- EDUCATIONAL_LAG: 15 post (5.7%)
- ACCESS TO SOCIAL SECURITY: 51 post (19.3%)
- HOUSING: 27 post (10.2%)
- ACCESS TO FOOD: 2 post (0.8%)
- SOCIAL_COHESION: 19 post (7.2%)
- non-poverty posts: 26 post (9.8%)

analyzing Yucatán (74 posts)...


Classifying Yucatán: 100%|██████████| 74/74 [00:02<00:00, 31.50it/s]



results for Yucatán:
total posts: 74

distribution of posts across dimensions of poverty:
- INCOME: 13 post (17.6%)
- ACCESS TO HEALTH SERVICES: 13 post (17.6%)
- EDUCATIONAL_LAG: 6 post (8.1%)
- ACCESS TO SOCIAL SECURITY: 12 post (16.2%)
- HOUSING: 11 post (14.9%)
- ACCESS TO FOOD: 9 post (12.2%)
- SOCIAL_COHESION: 6 post (8.1%)
- non-poverty posts: 4 post (5.4%)

analyzing Chiapas (219 posts)...


Classifying Chiapas: 100%|██████████| 219/219 [00:11<00:00, 19.37it/s]



results for Chiapas:
total posts: 219

distribution of posts across dimensions of poverty:
- INCOME: 26 post (11.9%)
- ACCESS TO HEALTH SERVICES: 64 post (29.2%)
- EDUCATIONAL_LAG: 25 post (11.4%)
- ACCESS TO SOCIAL SECURITY: 17 post (7.8%)
- HOUSING: 29 post (13.2%)
- ACCESS TO FOOD: 6 post (2.7%)
- SOCIAL_COHESION: 37 post (16.9%)
- non-poverty posts: 15 post (6.8%)

count of posts per dimension:
dimension            ACCESS TO FOOD  ACCESS TO HEALTH SERVICES  \
state                                                            
Aguascalientes                   12                         29   
Baja California                   1                         25   
Baja California Sur               1                          5   
Campeche                          0                         16   
Chiapas                           6                         64   
Chihuahua                        13                         47   
Ciudad de México                  1                        105   
Coa